# 📊 Factor Analysis for Portfolio Evaluation

**Understand what's really driving your portfolio returns**

This notebook demonstrates how to use factor models to:
- Decompose portfolio returns into systematic factors
- Measure alpha (skill) vs beta (market exposure)
- Identify hidden factor tilts in your portfolio
- Optimize portfolios with factor constraints

---

In [ ]:
# Install dependencies (required for Google Colab)
# Installing from GitHub to get latest version with factors module
!pip install git+https://github.com/engineerinvestor/Portfolio-Analysis.git pandas-datareader -q
print("✅ Dependencies installed!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set beautiful plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Color palette
COLORS = {
    'primary': '#2563eb',
    'secondary': '#7c3aed', 
    'success': '#059669',
    'danger': '#dc2626',
    'warning': '#d97706',
    'info': '#0891b2',
    'muted': '#6b7280',
    'market': '#3b82f6',
    'smb': '#8b5cf6',
    'hml': '#f59e0b',
    'mom': '#10b981',
    'alpha': '#ef4444'
}

print("✅ Setup complete!")

## 1. Load Portfolio & Factor Data

Let's analyze a sample portfolio and compare it against Fama-French factors.

In [ ]:
from portfolio_analysis import DataLoader, PortfolioAnalysis
from portfolio_analysis.factors import (
    FactorDataLoader, 
    FactorRegression,
    FactorAttribution,
    FactorExposures,
    FactorOptimizer
)

# Define analysis period
START_DATE = '2019-01-01'
END_DATE = '2024-01-01'

# Sample portfolio: Mix of growth, value, and bonds
TICKERS = ['VTI', 'VBR', 'VTV', 'VUG', 'BND']
WEIGHTS = [0.30, 0.15, 0.20, 0.20, 0.15]  # 30% Total Market, 15% Small Value, etc.

print("📥 Loading portfolio data...")
loader = DataLoader(TICKERS, START_DATE, END_DATE)
price_data = loader.fetch_data(progress=False)

# Calculate portfolio returns
portfolio = PortfolioAnalysis(price_data, WEIGHTS)
portfolio_returns = portfolio.calculate_portfolio_returns()

print(f"📅 Period: {START_DATE} to {END_DATE}")
print(f"📈 Portfolio: {dict(zip(TICKERS, WEIGHTS))}")
print(f"📊 {len(portfolio_returns)} trading days loaded")

In [ ]:
# Load Fama-French factor data
print("📥 Loading Fama-French factors...")
factor_loader = FactorDataLoader()

# Get different factor models
ff3_factors = factor_loader.get_ff3_factors(START_DATE, END_DATE, frequency='daily')
ff5_factors = factor_loader.get_ff5_factors(START_DATE, END_DATE, frequency='daily')
carhart_factors = factor_loader.get_carhart_factors(START_DATE, END_DATE, frequency='daily')

print(f"\n📊 Fama-French 3 Factors: {ff3_factors.columns.tolist()}")
print(f"📊 Fama-French 5 Factors: {ff5_factors.columns.tolist()}")
print(f"📊 Carhart 4 Factors: {carhart_factors.columns.tolist()}")

## 2. Factor Model Comparison

Let's see how well different factor models explain our portfolio returns.

In [ ]:
# Run regressions with different models
regression = FactorRegression(portfolio_returns, carhart_factors)

# Compare all models
comparison = regression.compare_models()
print(comparison.to_string(index=False))

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

models = comparison['Model'].tolist()
x = np.arange(len(models))

# R-squared comparison
ax = axes[0]
bars = ax.bar(x, comparison['R-squared'] * 100, color=COLORS['primary'], 
              edgecolor='white', linewidth=2, alpha=0.9)
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylabel('Explained Variance (%)')
ax.set_title('Model Fit (R²)')
ax.set_ylim(0, 100)
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 5), textcoords='offset points', ha='center', fontweight='bold')

# Alpha comparison
ax = axes[1]
alphas = comparison['Alpha (%)'].values
colors = [COLORS['success'] if a > 0 else COLORS['danger'] for a in alphas]
bars = ax.bar(x, alphas, color=colors, edgecolor='white', linewidth=2, alpha=0.9)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylabel('Annual Alpha (%)')
ax.set_title('Risk-Adjusted Excess Return')
for i, (bar, pval) in enumerate(zip(bars, comparison['Alpha p-value'])):
    height = bar.get_height()
    sig = '**' if pval < 0.01 else ('*' if pval < 0.05 else '')
    ax.annotate(f'{height:.2f}%{sig}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 5 if height >= 0 else -15), textcoords='offset points', 
                ha='center', fontweight='bold')

# Market Beta
ax = axes[2]
bars = ax.bar(x, comparison['Mkt Beta'], color=COLORS['market'], 
              edgecolor='white', linewidth=2, alpha=0.9)
ax.axhline(y=1, color=COLORS['danger'], linestyle='--', linewidth=2, label='Market (β=1)')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylabel('Market Beta')
ax.set_title('Market Exposure')
ax.legend(loc='upper right')
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 5), textcoords='offset points', ha='center', fontweight='bold')

plt.suptitle('Factor Model Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 3. Factor Exposures Deep Dive

Using the Fama-French 3-Factor model, let's examine our portfolio's factor exposures in detail.

In [ ]:
# Run FF3 regression
ff3_results = regression.run_regression('ff3')
print(ff3_results.summary())

In [ ]:
# Beautiful factor exposure visualization
fig, ax = plt.subplots(figsize=(10, 6))

factors = ['Mkt-RF', 'SMB', 'HML']
betas = [ff3_results.betas[f] for f in factors]
pvalues = [ff3_results.beta_pvalues[f] for f in factors]

# Color by significance and direction
colors = []
for beta, pval in zip(betas, pvalues):
    if pval >= 0.05:
        colors.append(COLORS['muted'])
    elif beta > 0:
        colors.append(COLORS['success'])
    else:
        colors.append(COLORS['danger'])

bars = ax.barh(factors, betas, color=colors, edgecolor='white', linewidth=2, height=0.6, alpha=0.9)
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.axvline(x=1, color=COLORS['muted'], linestyle='--', linewidth=1.5, alpha=0.7)

# Add value labels and significance stars
for bar, beta, pval in zip(bars, betas, pvalues):
    width = bar.get_width()
    sig = '***' if pval < 0.001 else ('**' if pval < 0.01 else ('*' if pval < 0.05 else ''))
    label = f'{beta:.3f}{sig}'
    ax.annotate(label, xy=(width, bar.get_y() + bar.get_height()/2),
                xytext=(10 if width >= 0 else -10, 0), textcoords='offset points',
                ha='left' if width >= 0 else 'right', va='center', 
                fontsize=14, fontweight='bold')

# Factor descriptions
descriptions = {
    'Mkt-RF': 'Market Risk Premium',
    'SMB': 'Small Minus Big (Size)',
    'HML': 'High Minus Low (Value)'
}

ax.set_yticks(range(len(factors)))
ax.set_yticklabels([f'{f}\n{descriptions[f]}' for f in factors], fontsize=11)
ax.set_xlabel('Factor Loading (Beta)', fontsize=12)
ax.set_title(f'Portfolio Factor Exposures\nR² = {ff3_results.r_squared:.1%} | Alpha = {ff3_results.alpha:.2%}/year', 
             fontsize=14, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=COLORS['success'], label='Positive (p<0.05)'),
    Patch(facecolor=COLORS['danger'], label='Negative (p<0.05)'),
    Patch(facecolor=COLORS['muted'], label='Not Significant')
]
ax.legend(handles=legend_elements, loc='lower right', framealpha=0.9)

ax.set_xlim(-0.5, max(betas) + 0.5)
plt.tight_layout()
plt.show()

## 4. Return Attribution: Where Did Returns Come From?

Decompose total portfolio returns into contributions from each factor.

In [ ]:
# Calculate attribution
attribution = FactorAttribution(portfolio_returns, ff3_factors)
return_decomp = attribution.decompose_returns('ff3')

print("\n📊 Return Attribution (Annualized)")
print("=" * 40)
for component, value in return_decomp.items():
    print(f"{component:15} {value*100:>8.2f}%")

In [ ]:
# Waterfall chart for return attribution
fig, ax = plt.subplots(figsize=(12, 7))

# Prepare data
components = ['Risk-Free', 'Mkt-RF', 'SMB', 'HML', 'Alpha']
values = [
    return_decomp['risk_free'] * 100,
    return_decomp['Mkt-RF'] * 100,
    return_decomp['SMB'] * 100,
    return_decomp['HML'] * 100,
    return_decomp['alpha'] * 100
]
total = return_decomp['total'] * 100

# Calculate cumulative positions for waterfall
cumulative = [0]
for v in values[:-1]:
    cumulative.append(cumulative[-1] + v)

# Colors
bar_colors = [
    COLORS['info'],      # Risk-free
    COLORS['market'],    # Market
    COLORS['smb'],       # SMB
    COLORS['hml'],       # HML
    COLORS['alpha']      # Alpha
]

# Draw waterfall bars
for i, (comp, val, cum, color) in enumerate(zip(components, values, cumulative, bar_colors)):
    bar = ax.bar(i, val, bottom=cum, color=color, edgecolor='white', 
                 linewidth=2, width=0.7, alpha=0.9)
    
    # Value label
    label_y = cum + val/2
    ax.annotate(f'{val:+.2f}%', xy=(i, label_y), ha='center', va='center',
                fontsize=12, fontweight='bold', color='white')

# Add total bar
ax.bar(len(components), total, color=COLORS['primary'], edgecolor='white', 
       linewidth=2, width=0.7, alpha=0.9)
ax.annotate(f'{total:.2f}%', xy=(len(components), total/2), ha='center', va='center',
            fontsize=14, fontweight='bold', color='white')

# Connecting lines
for i in range(len(values)):
    end_y = cumulative[i] + values[i]
    ax.plot([i + 0.35, i + 0.65], [end_y, end_y], color='gray', 
            linestyle='--', linewidth=1, alpha=0.5)

ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_xticks(range(len(components) + 1))
ax.set_xticklabels(components + ['Total'], fontsize=11, fontweight='bold')
ax.set_ylabel('Return Contribution (%)', fontsize=12)
ax.set_title('Return Attribution Waterfall\nDecomposing Portfolio Returns by Factor', 
             fontsize=14, fontweight='bold')

# Add descriptive annotations
ax.annotate('Systematic\nRisk Factors', xy=(2, ax.get_ylim()[1] * 0.9), 
            ha='center', fontsize=10, color=COLORS['muted'])

plt.tight_layout()
plt.show()

## 5. Rolling Factor Analysis: Exposures Over Time

Factor exposures aren't static - let's see how they've evolved.

In [ ]:
# Calculate rolling betas (60-day window = ~3 months)
rolling_betas = regression.run_rolling_regression('ff3', window=60)

print(f"📊 Rolling regression: {len(rolling_betas)} observations")
rolling_betas.tail()

In [ ]:
# Beautiful rolling factor visualization
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

factor_colors = {
    'Mkt-RF': COLORS['market'],
    'SMB': COLORS['smb'],
    'HML': COLORS['hml'],
    'alpha': COLORS['alpha']
}

factor_labels = {
    'Mkt-RF': 'Market Beta',
    'SMB': 'Size Factor (SMB)',
    'HML': 'Value Factor (HML)',
    'alpha': 'Rolling Alpha (Annualized)'
}

for ax, factor in zip(axes, ['Mkt-RF', 'SMB', 'HML', 'alpha']):
    color = factor_colors[factor]
    data = rolling_betas[factor] * (100 if factor == 'alpha' else 1)
    
    # Plot line
    ax.plot(rolling_betas.index, data, color=color, linewidth=1.5, alpha=0.9)
    
    # Fill area
    ax.fill_between(rolling_betas.index, data, alpha=0.2, color=color)
    
    # Reference line
    ref_val = 1 if factor == 'Mkt-RF' else 0
    ax.axhline(y=ref_val, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    # Mean line
    mean_val = data.mean()
    ax.axhline(y=mean_val, color=color, linestyle=':', linewidth=2, alpha=0.7)
    
    # Labels
    ax.set_ylabel(factor_labels[factor], fontsize=11, fontweight='bold')
    
    # Stats annotation
    stats_text = f'Mean: {mean_val:.3f}' if factor != 'alpha' else f'Mean: {mean_val:.2f}%'
    ax.annotate(stats_text, xy=(0.02, 0.95), xycoords='axes fraction',
                fontsize=10, fontweight='bold', color=color,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[0].set_title('Rolling Factor Exposures (60-Day Window)', fontsize=14, fontweight='bold')
axes[-1].set_xlabel('Date', fontsize=12)

# Add event markers (example: COVID crash)
for ax in axes:
    ax.axvline(x=pd.Timestamp('2020-03-23'), color='red', linestyle=':', 
               linewidth=1.5, alpha=0.7)

axes[0].annotate('COVID\nBottom', xy=(pd.Timestamp('2020-03-23'), axes[0].get_ylim()[1]),
                 xytext=(10, -10), textcoords='offset points', fontsize=9,
                 color='red', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Risk Attribution: Understanding Portfolio Volatility

Where does portfolio risk come from? Let's decompose variance by factor.

In [ ]:
# Risk decomposition
risk_decomp = attribution.decompose_risk('ff3')

print("\n📊 Risk Attribution")
print("=" * 40)
total_var = risk_decomp['total']
for component, value in risk_decomp.items():
    if component not in ['total', 'r_squared']:
        pct = value / total_var * 100 if total_var > 0 else 0
        print(f"{component:15} {pct:>8.1f}%")

In [ ]:
# Risk attribution donut chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Data
components = ['Mkt-RF', 'SMB', 'HML', 'Idiosyncratic']
values = [risk_decomp[c] for c in ['Mkt-RF', 'SMB', 'HML', 'idiosyncratic']]
total_var = risk_decomp['total']
percentages = [v / total_var * 100 for v in values]

colors = [COLORS['market'], COLORS['smb'], COLORS['hml'], COLORS['muted']]

# Donut chart
ax = axes[0]
wedges, texts, autotexts = ax.pie(
    percentages, 
    labels=components,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.75,
    explode=[0.02] * 4,
    wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2)
)
for autotext in autotexts:
    autotext.set_fontweight('bold')
    autotext.set_fontsize(11)

# Center text
ax.text(0, 0, f'R²\n{risk_decomp["r_squared"]:.1%}', 
        ha='center', va='center', fontsize=16, fontweight='bold')
ax.set_title('Variance Decomposition', fontsize=14, fontweight='bold')

# Horizontal bar chart
ax = axes[1]
y_pos = np.arange(len(components))
bars = ax.barh(y_pos, percentages, color=colors, edgecolor='white', 
               linewidth=2, height=0.6, alpha=0.9)

ax.set_yticks(y_pos)
ax.set_yticklabels(components, fontsize=11)
ax.set_xlabel('Contribution to Total Variance (%)', fontsize=12)
ax.set_title('Risk Factor Contributions', fontsize=14, fontweight='bold')

for bar, pct in zip(bars, percentages):
    ax.annotate(f'{pct:.1f}%', xy=(bar.get_width(), bar.get_y() + bar.get_height()/2),
                xytext=(5, 0), textcoords='offset points', 
                ha='left', va='center', fontweight='bold', fontsize=11)

ax.set_xlim(0, max(percentages) * 1.2)

plt.suptitle(f'Portfolio Risk Attribution\nTotal Volatility: {np.sqrt(total_var * 252) * 100:.1f}% annualized',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 7. Characteristic-Based Factor Tilts

Beyond regression, we can estimate factor exposures from portfolio characteristics.

In [ ]:
# Calculate characteristic-based tilts
exposures = FactorExposures(TICKERS, WEIGHTS)
tilts = exposures.get_all_tilts()

print("\n📊 Characteristic-Based Factor Tilts")
print("=" * 40)
for factor, tilt in tilts.items():
    direction = '↑' if tilt > 0.1 else ('↓' if tilt < -0.1 else '→')
    print(f"{factor.capitalize():15} {tilt:>6.2f}  {direction}")

In [ ]:
# Radar chart for factor tilts
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

factors = list(tilts.keys())
values = list(tilts.values())

# Close the plot
angles = np.linspace(0, 2 * np.pi, len(factors), endpoint=False).tolist()
values_plot = values + [values[0]]
angles_plot = angles + [angles[0]]

# Plot
ax.plot(angles_plot, values_plot, 'o-', linewidth=3, markersize=10, 
        color=COLORS['primary'], alpha=0.9)
ax.fill(angles_plot, values_plot, alpha=0.25, color=COLORS['primary'])

# Reference circles
for val in [-1, -0.5, 0, 0.5, 1]:
    ax.plot(angles_plot, [val] * len(angles_plot), '--', 
            color='gray', alpha=0.3, linewidth=0.5)

# Labels
factor_labels = {
    'size': 'Size\n(Small → Large)',
    'value': 'Value\n(Growth → Value)', 
    'momentum': 'Momentum\n(Low → High)',
    'quality': 'Quality\n(Low → High)',
    'investment': 'Investment\n(Aggressive → Conservative)'
}

ax.set_xticks(angles)
ax.set_xticklabels([factor_labels[f] for f in factors], fontsize=11, fontweight='bold')
ax.set_ylim(-1.2, 1.2)

# Add value annotations
for angle, factor, value in zip(angles, factors, values):
    ax.annotate(f'{value:.2f}', xy=(angle, value), xytext=(0, 10),
                textcoords='offset points', ha='center', fontsize=11, 
                fontweight='bold', color=COLORS['primary'])

ax.set_title('Portfolio Factor Tilts\n(Characteristic-Based Analysis)', 
             fontsize=14, fontweight='bold', y=1.08)

plt.tight_layout()
plt.show()

## 8. Individual Asset Factor Exposures

Let's examine how each asset contributes to overall factor exposure.

In [ ]:
# Calculate betas for each individual asset
asset_returns = price_data.pct_change().dropna()

asset_betas = {}
for ticker in TICKERS:
    reg = FactorRegression(asset_returns[ticker], ff3_factors)
    results = reg.run_regression('ff3')
    asset_betas[ticker] = results.betas

asset_betas_df = pd.DataFrame(asset_betas).T
asset_betas_df['Weight'] = WEIGHTS
print(asset_betas_df.round(3))

In [ ]:
# Stacked bar chart showing factor contributions by asset
fig, ax = plt.subplots(figsize=(12, 7))

factors = ['Mkt-RF', 'SMB', 'HML']
x = np.arange(len(TICKERS))
width = 0.25

colors = [COLORS['market'], COLORS['smb'], COLORS['hml']]

for i, (factor, color) in enumerate(zip(factors, colors)):
    values = [asset_betas[t][factor] for t in TICKERS]
    bars = ax.bar(x + i * width, values, width, label=factor, 
                  color=color, edgecolor='white', linewidth=1.5, alpha=0.9)

ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.axhline(y=1, color=COLORS['muted'], linestyle='--', linewidth=1.5, alpha=0.5)

ax.set_xticks(x + width)
ax.set_xticklabels([f'{t}\n({w:.0%})' for t, w in zip(TICKERS, WEIGHTS)], fontsize=11)
ax.set_ylabel('Factor Beta', fontsize=12)
ax.set_title('Factor Exposures by Asset\n(with portfolio weights)', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', framealpha=0.9)

# Add asset descriptions
asset_desc = {
    'VTI': 'Total Market',
    'VBR': 'Small Value',
    'VTV': 'Large Value',
    'VUG': 'Large Growth',
    'BND': 'Total Bond'
}

for i, ticker in enumerate(TICKERS):
    ax.annotate(asset_desc[ticker], xy=(i + width, ax.get_ylim()[0]),
                xytext=(0, -25), textcoords='offset points',
                ha='center', fontsize=9, color=COLORS['muted'])

plt.tight_layout()
plt.show()

## 9. Factor-Aware Portfolio Optimization

Optimize portfolio weights to achieve specific factor targets.

In [ ]:
# Create factor optimizer
optimizer = FactorOptimizer(price_data, ff3_factors)

# Current portfolio betas
print("\n📊 Current Portfolio Factor Exposures:")
print(f"   Market Beta: {ff3_results.betas['Mkt-RF']:.3f}")
print(f"   SMB (Size):  {ff3_results.betas['SMB']:.3f}")
print(f"   HML (Value): {ff3_results.betas['HML']:.3f}")

In [ ]:
# Optimize for different factor targets
scenarios = {
    'Market Neutral': {'Mkt-RF': 0.0},
    'Defensive (Low Beta)': {'Mkt-RF': 0.6},
    'Value Tilt': {'Mkt-RF': 1.0, 'HML': 0.4},
    'Small Cap Tilt': {'Mkt-RF': 1.0, 'SMB': 0.5},
}

results = {}
for name, targets in scenarios.items():
    try:
        result = optimizer.optimize_target_exposures(targets, tolerance=0.15)
        if result['success']:
            results[name] = result
            print(f"\n✅ {name}:")
            for t, w in result['weights'].items():
                if w > 0.01:
                    print(f"   {t}: {w:.1%}")
    except Exception as e:
        print(f"❌ {name}: Could not optimize")

In [ ]:
# Visualize optimization results
if results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Weights comparison
    ax = axes[0]
    scenario_names = list(results.keys())
    x = np.arange(len(TICKERS))
    width = 0.8 / len(scenario_names)
    
    colors_list = [COLORS['primary'], COLORS['secondary'], COLORS['success'], COLORS['warning']]
    
    for i, (name, result) in enumerate(results.items()):
        weights = [result['weights'].get(t, 0) * 100 for t in TICKERS]
        ax.bar(x + i * width, weights, width, label=name, 
               color=colors_list[i % len(colors_list)], alpha=0.8)
    
    ax.set_xticks(x + width * (len(scenario_names) - 1) / 2)
    ax.set_xticklabels(TICKERS, fontsize=11)
    ax.set_ylabel('Weight (%)', fontsize=12)
    ax.set_title('Optimal Weights by Strategy', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    
    # Risk-return comparison
    ax = axes[1]
    for i, (name, result) in enumerate(results.items()):
        ax.scatter(result['volatility'] * 100, result['return'] * 100,
                   s=200, c=colors_list[i % len(colors_list)], 
                   edgecolor='white', linewidth=2, label=name, alpha=0.9)
        ax.annotate(name, xy=(result['volatility'] * 100, result['return'] * 100),
                    xytext=(10, 5), textcoords='offset points', fontsize=9)
    
    ax.set_xlabel('Volatility (%)', fontsize=12)
    ax.set_ylabel('Expected Return (%)', fontsize=12)
    ax.set_title('Risk-Return Trade-off', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 10. Summary Dashboard

A comprehensive view of portfolio factor analysis.

In [ ]:
# Create summary dashboard
fig = plt.figure(figsize=(16, 12))

# Layout: 3x3 grid
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)

# 1. Portfolio composition (top left)
ax1 = fig.add_subplot(gs[0, 0])
colors_pie = plt.cm.Set3(np.linspace(0, 1, len(TICKERS)))
wedges, texts, autotexts = ax1.pie(WEIGHTS, labels=TICKERS, autopct='%1.0f%%',
                                    colors=colors_pie, startangle=90,
                                    wedgeprops=dict(edgecolor='white', linewidth=2))
ax1.set_title('Portfolio Allocation', fontsize=12, fontweight='bold')

# 2. Key metrics (top center)
ax2 = fig.add_subplot(gs[0, 1])
ax2.axis('off')
metrics_text = f"""
    Annual Return:     {portfolio.calculate_portfolio_return()*100:.1f}%
    Annual Volatility: {portfolio.calculate_portfolio_volatility()*100:.1f}%
    Sharpe Ratio:      {portfolio.calculate_portfolio_sharpe_ratio():.2f}
    Max Drawdown:      {portfolio.calculate_max_drawdown()*100:.1f}%
    
    Market Beta:       {ff3_results.betas['Mkt-RF']:.3f}
    Alpha (annual):    {ff3_results.alpha*100:.2f}%
    R-squared:         {ff3_results.r_squared:.1%}
"""
ax2.text(0.1, 0.5, metrics_text, transform=ax2.transAxes, fontsize=12,
         verticalalignment='center', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax2.set_title('Key Metrics', fontsize=12, fontweight='bold')

# 3. Factor betas (top right)
ax3 = fig.add_subplot(gs[0, 2])
factors = ['Mkt-RF', 'SMB', 'HML']
betas = [ff3_results.betas[f] for f in factors]
colors_bar = [COLORS['market'], COLORS['smb'], COLORS['hml']]
ax3.barh(factors, betas, color=colors_bar, edgecolor='white', linewidth=2, alpha=0.9)
ax3.axvline(x=0, color='black', linewidth=1)
ax3.axvline(x=1, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax3.set_xlabel('Beta')
ax3.set_title('Factor Exposures', fontsize=12, fontweight='bold')
for i, beta in enumerate(betas):
    ax3.annotate(f'{beta:.3f}', xy=(beta, i), xytext=(5, 0),
                 textcoords='offset points', va='center', fontweight='bold')

# 4. Return attribution (middle, spanning 2 cols)
ax4 = fig.add_subplot(gs[1, :2])
components = ['Risk-Free', 'Market', 'Size', 'Value', 'Alpha']
values = [
    return_decomp['risk_free'] * 100,
    return_decomp['Mkt-RF'] * 100,
    return_decomp['SMB'] * 100,
    return_decomp['HML'] * 100,
    return_decomp['alpha'] * 100
]
colors_attr = [COLORS['info'], COLORS['market'], COLORS['smb'], COLORS['hml'], COLORS['alpha']]
bars = ax4.bar(components, values, color=colors_attr, edgecolor='white', linewidth=2, alpha=0.9)
ax4.axhline(y=0, color='black', linewidth=1)
ax4.set_ylabel('Contribution (%)')
ax4.set_title(f'Return Attribution (Total: {return_decomp["total"]*100:.1f}%)', 
              fontsize=12, fontweight='bold')
for bar, val in zip(bars, values):
    ax4.annotate(f'{val:+.2f}%', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                 xytext=(0, 5 if val >= 0 else -15), textcoords='offset points',
                 ha='center', fontweight='bold', fontsize=10)

# 5. Risk attribution pie (middle right)
ax5 = fig.add_subplot(gs[1, 2])
risk_components = ['Market', 'Size', 'Value', 'Specific']
risk_values = [
    risk_decomp['Mkt-RF'] / risk_decomp['total'] * 100,
    risk_decomp['SMB'] / risk_decomp['total'] * 100,
    risk_decomp['HML'] / risk_decomp['total'] * 100,
    risk_decomp['idiosyncratic'] / risk_decomp['total'] * 100
]
colors_risk = [COLORS['market'], COLORS['smb'], COLORS['hml'], COLORS['muted']]
ax5.pie(risk_values, labels=risk_components, autopct='%1.0f%%',
        colors=colors_risk, startangle=90,
        wedgeprops=dict(width=0.6, edgecolor='white', linewidth=2))
ax5.set_title('Risk Attribution', fontsize=12, fontweight='bold')

# 6. Rolling market beta (bottom, spanning all cols)
ax6 = fig.add_subplot(gs[2, :])
ax6.plot(rolling_betas.index, rolling_betas['Mkt-RF'], 
         color=COLORS['market'], linewidth=1.5, alpha=0.9)
ax6.fill_between(rolling_betas.index, rolling_betas['Mkt-RF'], alpha=0.2, color=COLORS['market'])
ax6.axhline(y=1, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, label='Market')
ax6.axhline(y=rolling_betas['Mkt-RF'].mean(), color=COLORS['market'], 
            linestyle=':', linewidth=2, alpha=0.7, label=f'Mean ({rolling_betas["Mkt-RF"].mean():.2f})')
ax6.set_xlabel('Date')
ax6.set_ylabel('Market Beta')
ax6.set_title('Rolling Market Beta (60-Day Window)', fontsize=12, fontweight='bold')
ax6.legend(loc='upper right')

fig.suptitle('Portfolio Factor Analysis Dashboard', fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

---

## Key Takeaways

### What Factor Analysis Reveals:

1. **Market Exposure (Beta)**: How much of your returns come from general market movements
2. **Size Tilt (SMB)**: Whether you're tilted toward small or large cap stocks
3. **Value Tilt (HML)**: Whether you're tilted toward value or growth stocks
4. **Alpha**: The return you generate beyond what factors explain (manager skill or luck)

### Practical Applications:

- **Performance Evaluation**: Is your "alpha" real or just hidden factor exposure?
- **Risk Management**: Understand where your portfolio volatility comes from
- **Portfolio Construction**: Target specific factor exposures
- **Fee Justification**: Are you paying active fees for passive factor exposure?

---

*Built with [Portfolio Analysis](https://github.com/engineerinvestor/Portfolio-Analysis) by Engineer Investor*